In [20]:
import numpy as np
from scipy.stats import norm
from skopt import Optimizer
import random


seed_int = 0
random.seed(seed_int)


def objective_function(X_list):
    sum=0
    for item in X_list:
        sum+=item
    return sum


def bayesian_optimization(bounds, iterations):

    #presets
    optimizer = Optimizer(
        dimensions=bounds,
        base_estimator="gp",
        acq_func="EI",
        acq_optimizer="sampling",
        random_state=seed_int,
    )



    X_init =[np.random.uniform(item[0], item[1]) for item in bounds]
    Y_init = objective_function(X_init)


    optimizer.tell(X_init, Y_init)


    for _ in range(iterations):
        x_next = optimizer.ask()
        print(x_next)
        y_next = objective_function(x_next)
        optimizer.tell(x_next, y_next)

    #LOOK: does the optimizer shoot for the min or the max?
    #gpt: the goal is to minimize the obejective function...

    sorted_pairs = sorted(zip(optimizer.Xi, optimizer.yi), key = lambda pair : pair[1], reverse=False)

    X = sorted_pairs[0][0]
    y = sorted_pairs[0][1]

    return X, y



bounds = [(-10.0, 5.0),(-10.0, 5.0), (-10.0, 5.0), (-10.0, 5.0)]


X, y = bayesian_optimization(bounds, 20)

print("Solution: x", X)
print("Result: y", y)

[-1.107330726624724, 2.663986228715263, 2.8691842643413548, 2.7087760817618847]
[-0.6465445482104126, -4.234274390609501, -5.536980901832915, -9.149305340238351]
[-5.9101555812983015, -2.8350232401797504, 2.1825309316324013, -2.8003424143741382]
[-4.108228058487552, 2.5411814530606662, -4.939057593740974, -0.2774219192320402]
[-4.476376902391779, 4.357327384295699, -7.894738293810322, 3.0513088753765487]
[-2.895879320894341, 2.0136612796946665, -2.1928378067319256, 0.18319295178440598]
[0.8094898208887535, -1.269703118873391, -1.939401558264839, 1.3792343648353604]
[-8.411385892183118, -2.8959937098001367, -7.2050148500986, 1.053772656934374]
[-6.751744683634421, -7.971727398918219, -5.137884883101788, -7.754876992244752]
[-9.309451333370031, -9.758257402544345, -9.484266654211075, -9.343185623793731]
[-9.735787347648929, -9.208698724011317, -9.829504333367563, -8.051499431193394]
[-9.857420938941116, -9.74929545129616, -9.974346243529098, -7.492007059702901]
[-9.67832954156221, -9.341